In [1]:
!pip install -U mittens

In [28]:
import csv
import numpy as np
import json
import re
import nltk
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ',quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed
glove_path = "ubercorpus.lowercased.tokenized.glove.300d"
pre_glove = glove2dict(glove_path)

In [3]:
list(pre_glove.items())[0]

('в',
 array([ 9.836460e-01,  3.129690e-01,  8.787120e-01,  4.810430e-01,
        -2.713210e-01, -5.224100e-02,  3.847660e-01,  5.488840e-01,
         6.209010e-01, -1.556510e-01, -2.214990e-01, -7.702480e-01,
         4.522260e-01,  5.103490e-01, -8.448350e-01, -4.302850e-01,
         7.468280e-01,  4.674520e-01, -9.083710e-01, -2.391460e-01,
        -3.792460e-01,  7.073960e-01,  1.363500e-01,  5.426200e-02,
         6.083940e-01, -4.272420e-01,  4.377500e-01,  5.590210e-01,
        -9.063890e-01, -2.274300e-02, -2.922630e-01,  3.069720e-01,
        -7.020210e-01, -5.330760e-01, -3.874640e-01,  4.564430e-01,
         2.494770e-01, -9.005080e-01, -2.404620e-01, -3.124110e-01,
         3.936140e-01, -6.756130e-01, -5.416980e-01, -6.663750e-01,
        -3.309430e-01,  5.328360e-01, -1.481300e-02,  1.659600e-02,
         5.994540e-01,  3.283400e-02, -1.470930e-01,  2.744950e-01,
         8.060810e-01, -3.953380e-01,  5.299290e-01, -1.471790e-01,
        -3.718410e-01,  2.654040e-01, -3.7

In [13]:
path = './'

with open(path + 'telegram/egege/result.json', encoding="utf-8") as f1:
    data_egege = json.load(f1)

with open(path + 'telegram/cs_ba/result.json', encoding="utf-8") as f2:
    data_cs_ba = json.load(f2)

with open(path + 'telegram/tovarystvo_dumky/result.json', encoding="utf-8") as f3:
    data_tovarystvo_dumky = json.load(f3)

with open(path + 'telegram/ucu_random/result.json', encoding="utf-8") as f4:
    data_ucu_random = json.load(f4)

with open(path + 'telegram/lnu/result.json', encoding="utf-8") as f5:
    data_lnu = json.load(f5)

with open(path + 'telegram/danka_the_best/result.json', encoding="utf-8") as f6:
    data_danka_the_best = json.load(f6)

corpus = []

messages = data_egege['messages'] + data_cs_ba['messages'] + data_tovarystvo_dumky['messages']\
           + data_ucu_random['messages'] + data_lnu['messages'] + data_danka_the_best['messages']

for message in messages:
    if isinstance(message['text'], str):
        text = re.sub(r'[,!?;-]+', '.', message['text'])
        corpus.append(nltk.word_tokenize(text))

print(len(corpus))

252689


In [15]:
corpus[1]

['Ви', 'уже', 'робили', 'презентацію', '.']

In [20]:
corpus_words = []

for sent in corpus:
    for word in sent:
        corpus_words.append(word)

In [21]:
corpus_words

['Ви',
 'уже',
 'робили',
 'презентацію',
 '.',
 'Нє',
 'Вже',
 'зробили',
 'Там',
 'в',
 'тараса',
 'ще',
 'вайтборд',
 'Ок',
 'Або',
 'вже',
 'закінчився',
 'Я',
 'тоді',
 'зараз',
 'підійду',
 'А',
 'ви',
 'де',
 '.',
 'ну',
 'я',
 'щас',
 'в',
 'цш',
 'так',
 'го',
 'в',
 'цш',
 'там',
 'якраз',
 '402',
 'вільна',
 'Прям',
 'щас',
 '.',
 'ну',
 'да',
 'Тарас',
 'вільний',
 '.',
 'Я',
 'речі',
 'збираю',
 'Давайте',
 'док',
 'А',
 'я',
 'підхожу',
 'Ви',
 'почніть',
 'в',
 'доку',
 'А',
 'ми',
 'доєднаємось',
 'Ви',
 'де',
 '.',
 'Я',
 'в',
 'уку',
 'Куди',
 'йти',
 '.',
 'можна',
 'в',
 'цш',
 '.',
 'можна',
 'в',
 'нас',
 'в',
 'кімнаті',
 'макс',
 'ти',
 'де',
 '.',
 'В',
 'параші',
 'В',
 'нас',
 'в',
 'кімнаті',
 'так',
 'а',
 'ти',
 'ще',
 'довго',
 'будеш',
 '.',
 'Я',
 'через',
 '20',
 'хв',
 'виїжаю',
 'Я',
 'йду',
 'о',
 'так',
 '20хв',
 'це',
 'ще',
 'норм',
 'го',
 'тоді',
 'до',
 'нас',
 'в',
 'кімнату',
 'Ок',
 'Пустіть',
 'тільки',
 'Реєстрація',
 'і',
 'аналіз',
 'ЕЕ

In [22]:
oov = [token.lower() for token in corpus_words if token.lower() not in pre_glove.keys()]

In [25]:
def get_rareoov(xdict, val):
    return [k for (k,v) in Counter(xdict).items() if v<=val]
oov_rare = get_rareoov(oov, 1)
corp_vocab = list(set(oov) - set(oov_rare))

In [26]:
brown_tokens = [token for token in oov if token not in oov_rare]
brown_doc = [' '.join(brown_tokens)]
corp_vocab = list(set(oov))

In [29]:
cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(brown_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

C:\Users\User\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:708: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self[i, j] = values


MemoryError: 